In [1]:
import re,os,csv,time,urllib,pylrc,shutil,requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, date, timedelta

In [11]:
# (9)匯入歌單(df_songlist.xlsx)
def import_songlist():
    df_songlist = pd.read_excel(r'table/singer_101-104/6_df_songlist.xlsx')
    return df_songlist

In [3]:
# (10)搜尋歌詞
# https://github.com/cjlh/spotify-lyrics/blob/master/spotify-lyrics.py
def print_lyrics(artist,title):
    pageurl = "https://makeitpersonal.co/lyrics?artist=" + artist + "&title=" + title
    lyrics = requests.get(pageurl).text.strip()  
    # 如果找不到歌詞的方案
    try:
        if lyrics == "Sorry, We don't have lyrics for this song yet.":
            wiki_url = "https://lyrics.fandom.com/wiki/"
            title = title.replace(" ", "_")
            artist = artist.replace(" ", "_")
            url = wiki_url + f"{artist}:{title}"

            r = requests.get(url)
            soup = BeautifulSoup(r.text, 'html.parser')

            lyric_box = str(soup.find("div", {"class": "lyricbox"}))
            lyrics = lyric_box.replace("<br/>", "\n")
            lyrics = lyrics.replace('<div class="lyricbox">', '')
            lyrics = lyrics.replace('<div class="lyricsbreak">', '')
            lyrics = lyrics.replace('</div>', '')
    except:
        print('None Lyrics')
        check = 1
        return check
        pass
    return lyrics

In [12]:
# (11)歌詞儲存成txt檔
def save2txt(artist,title,lyrics):
    filename =  'lyrics/singer_101-104/' + artist + '_' + title + '.txt'
    f = open(filename,'w',encoding="utf-8")
    try:
        f.write(lyrics)
    except:
        pass
    return filename

In [13]:
# (12)判斷歌詞txt是否為None，若為None，則複製txt檔案到令一個資料夾(lyrics_none)
def check_none(artist,title,lyrics):
    if lyrics == 'None':
        check = 0 # 0代表沒有歌詞
        base_lyrics = 'D:/Anaconda3/Scripts/7 實驗進度/SpotifyLyrics/lyrics/singer_101-104/'
        base_lyrics_none = 'D:/Anaconda3/Scripts/7 實驗進度/SpotifyLyrics/lyrics_none/singer_101-104/'
        x = base_lyrics + artist + '_' + title + '.txt'
        #print(x)
        move_x = base_lyrics_none + artist + '_' + title + '.txt'
        shutil.move(x,move_x) # 若lyrics回傳None，就把歌詞txt檔案移到lyrics_none資料夾中
    else:
        check = 1 # 1代表有歌詞
    return check

In [15]:
# 主程式下載歌詞
df_songlist = import_songlist()
count = 0
list_with_lyrics = [] # 存放有回傳歌詞的索引值的陣列

for i in range (len(df_songlist)): #len(df_songlist)
    artist = df_songlist['Artist'][i]
    title = df_songlist['Title'][i]
    lyrics = print_lyrics(artist,title)
    filename = save2txt(artist,title,lyrics)
    check = check_none(artist,title,lyrics)
    print(str(check) + ' - Filename: '+ filename)
    #print('Lyrics:'+ '\n' + lyrics)
    try:
        if check == 1: # 1代表有歌詞
            count += 1
            list_with_lyrics.append(i)
        else:
            pass
    except:
        pass

print('Count:',count)
print(list_with_lyrics)

0 - Filename: lyrics/singer_101-104/Tori Kelly_12 16 1992.txt
0 - Filename: lyrics/singer_101-104/Tori Kelly_2 Places.txt
0 - Filename: lyrics/singer_101-104/Tori Kelly_3 2 1991.txt
0 - Filename: lyrics/singer_101-104/Tori Kelly_3 26 1994.txt
0 - Filename: lyrics/singer_101-104/Tori Kelly_8 28 1997.txt
0 - Filename: lyrics/singer_101-104/Tori Kelly_Actress.txt
1 - Filename: lyrics/singer_101-104/Tori Kelly_Anyway.txt
1 - Filename: lyrics/singer_101-104/Tori Kelly_Art Of Letting You Go.txt
0 - Filename: lyrics/singer_101-104/Tori Kelly_Before The Dawn.txt
1 - Filename: lyrics/singer_101-104/Tori Kelly_California Lovers.txt
0 - Filename: lyrics/singer_101-104/Tori Kelly_Change Your Mind.txt
1 - Filename: lyrics/singer_101-104/Tori Kelly_City Dove.txt
0 - Filename: lyrics/singer_101-104/Tori Kelly_Coffee.txt
1 - Filename: lyrics/singer_101-104/Tori Kelly_Daydream.txt
1 - Filename: lyrics/singer_101-104/Tori Kelly_Dear No One.txt
1 - Filename: lyrics/singer_101-104/Tori Kelly_Expensive.txt

In [16]:
len(list_with_lyrics)

69

In [17]:
# (13)存在於df_songlist的歌曲(代表有下載到歌詞的)，複製歌曲到另一個資料夾(songs_edit)並依據「歌手_歌名」格式重新命名
def copyfile(base1,base2,df_songlist,list_with_lyrics):
    oldname_list = [] # 在songs2資料夾的原始檔名
    currentname_list = [] # 複製到songs_edit資料夾的檔名
    newname_list = [] # 欲修改的新檔名
    for i in range(len(list_with_lyrics)):
        index = list_with_lyrics[i]
        original = df_songlist['Original'][index]
        artist = df_songlist['Artist'][index]
        title = df_songlist['Title'][index]
        
        filename = title + '.mp3'
        #base1 = r'D:/TuneFab Spotify Music Converter/songs2/'
        oldpath = base1 + original + '/' + filename # 原始音檔位址
        oldname_list.append(oldpath)
        #base2 = r'D:/TuneFab Spotify Music Converter/songs_edit/'
        newpath = base2 + filename # 複製目的地
        currentname_list.append(newpath)
        
        newname = base2 + artist + '_' + filename # 複製後的新檔名
        newname_list.append(newname)
        
        if os.path.exists(oldpath): # 音檔存在
            message = 'exists'
            shutil.copy(oldpath,newpath) # 複製檔案
            #print(message + ' - ' + newpath)
            #print(newpath + ' >>> ' + newname)
            os.rename(newpath,newname) # 重新命名
        else:
            message = 'not exist' # 音檔不存在
            #print(message)
    
    df_copyfile = pd.DataFrame(list(zip(oldname_list,currentname_list,newname_list)),columns=['OldName','CurrentName','NewName'])
    return df_copyfile

In [18]:
base1 = r'D:/TuneFab Spotify Music Converter/songs2/singer_101-104/'
base2 = r'D:/TuneFab Spotify Music Converter/songs_edit/singer_101-104/'
df_copyfile = copyfile(base1,base2,df_songlist,list_with_lyrics)
df_copyfile

,OldName,CurrentName,NewName
0,D:/TuneFab Spotify Music Converter/songs2/sing...,D:/TuneFab Spotify Music Converter/songs_edit/...,D:/TuneFab Spotify Music Converter/songs_edit/...
1,D:/TuneFab Spotify Music Converter/songs2/sing...,D:/TuneFab Spotify Music Converter/songs_edit/...,D:/TuneFab Spotify Music Converter/songs_edit/...
2,D:/TuneFab Spotify Music Converter/songs2/sing...,D:/TuneFab Spotify Music Converter/songs_edit/...,D:/TuneFab Spotify Music Converter/songs_edit/...
3,D:/TuneFab Spotify Music Converter/songs2/sing...,D:/TuneFab Spotify Music Converter/songs_edit/...,D:/TuneFab Spotify Music Converter/songs_edit/...
4,D:/TuneFab Spotify Music Converter/songs2/sing...,D:/TuneFab Spotify Music Converter/songs_edit/...,D:/TuneFab Spotify Music Converter/songs_edit/...
...,...,...,...
64,D:/TuneFab Spotify Music Converter/songs2/sing...,D:/TuneFab Spotify Music Converter/songs_edit/...,D:/TuneFab Spotify Music Converter/songs_edit/...
65,D:/TuneFab Spotify Music Converter/songs2/sing...,D:/TuneFab Spotify Music Converter/songs_edit/...,D:/TuneFab Spotify Music Converter/songs_edit/...
66,D:/TuneFab Spotify Music Converter/songs2/sing...,D:/TuneFab Spotify Music Converter/songs_edit/...,D:/TuneFab Spotify Music Converter/songs_edit/...
67,D:/TuneFab Spotify Music Converter/songs2/sing...,D:/TuneFab Spotify Music Converter/songs_edit/...,D:/TuneFab Spotify Music Converter/songs_edit/...


 ___

In [50]:
# (13)存在於df_songlist的歌曲(代表有下載到歌詞的)，複製歌曲到另一個資料夾(songs_edit)
# 修改：複製檔案後直接重新命名(防止不同歌手同歌名的情況發生)
def copyfile_test(base1_test,base2_test,df_songlist,list_with_lyrics):
    oldname_list = [] # 在songs2資料夾的原始檔名
    currentname_list = [] # 複製到songs_edit資料夾的檔名
    newname_list = [] # 欲修改的新檔名
    for i in range(len(list_with_lyrics)):
        index = list_with_lyrics[i]
        original = df_songlist['Original'][index]
        artist = df_songlist['Artist'][index]
        title = df_songlist['Title'][index]
        
        filename = title + '.mp3'
        #base1 = r'D:/TuneFab Spotify Music Converter/songs2/'
        oldpath = base1_test + original + '/' + filename # 原始音檔位址
        oldname_list.append(oldpath)
        #base2 = r'D:/TuneFab Spotify Music Converter/songs_edit/'
        newpath = base2_test + filename # 複製目的地
        currentname_list.append(newpath)
        
        newname = base2_test + artist + '_' + filename # 複製後的新檔名，後面renamefile()執行會使用到
        newname_list.append(newname)
        
        if os.path.exists(oldpath): # 音檔存在
            message = 'exists'
            shutil.copy(oldpath,newpath)
            #print(message + ' - ' + newpath)
            #print(newpath + ' >>> ' + newname)
            os.rename(newpath,newname)
        else:
            message = 'not exist' # 音檔不存在
            #print(message)
    
    df_copyfile_test = pd.DataFrame(list(zip(oldname_list,currentname_list,newname_list)),columns=['OldName','CurrentName','NewName'])
    return df_copyfile_test

In [51]:
base1_test = r'D:/TuneFab Spotify Music Converter/songs2/singer_041-060/'
base2_test = r'C:/Users/jenny/Desktop/copyfile_test_folder/'
df_copyfile_test = copyfile_test(base1_test,base2_test,df_songlist,list_with_lyrics)
df_copyfile_test

,OldName,CurrentName,NewName
0,D:/TuneFab Spotify Music Converter/songs2/sing...,C:/Users/jenny/Desktop/copyfile_test_folder/A ...,C:/Users/jenny/Desktop/copyfile_test_folder/Gr...
1,D:/TuneFab Spotify Music Converter/songs2/sing...,C:/Users/jenny/Desktop/copyfile_test_folder/Be...,C:/Users/jenny/Desktop/copyfile_test_folder/Gr...
2,D:/TuneFab Spotify Music Converter/songs2/sing...,C:/Users/jenny/Desktop/copyfile_test_folder/Bu...,C:/Users/jenny/Desktop/copyfile_test_folder/Gr...
3,D:/TuneFab Spotify Music Converter/songs2/sing...,C:/Users/jenny/Desktop/copyfile_test_folder/Ci...,C:/Users/jenny/Desktop/copyfile_test_folder/Gr...
4,D:/TuneFab Spotify Music Converter/songs2/sing...,C:/Users/jenny/Desktop/copyfile_test_folder/Cl...,C:/Users/jenny/Desktop/copyfile_test_folder/Gr...
...,...,...,...
502,D:/TuneFab Spotify Music Converter/songs2/sing...,C:/Users/jenny/Desktop/copyfile_test_folder/Ve...,C:/Users/jenny/Desktop/copyfile_test_folder/La...
503,D:/TuneFab Spotify Music Converter/songs2/sing...,C:/Users/jenny/Desktop/copyfile_test_folder/We...,C:/Users/jenny/Desktop/copyfile_test_folder/La...
504,D:/TuneFab Spotify Music Converter/songs2/sing...,C:/Users/jenny/Desktop/copyfile_test_folder/Wh...,C:/Users/jenny/Desktop/copyfile_test_folder/La...
505,D:/TuneFab Spotify Music Converter/songs2/sing...,C:/Users/jenny/Desktop/copyfile_test_folder/Wh...,C:/Users/jenny/Desktop/copyfile_test_folder/La...


In [39]:
# (14)資料夾(songs_edit)的歌曲重新命名，依據「歌手_歌名」格式命名
def renamefile(df_copyfile):
    for i in range(len(df_copyfile)):
        old = df_copyfile['OldName'][i]
        current = df_copyfile['CurrentName'][i]
        new = df_copyfile['NewName'][i]
        try:
            os.rename(current,new)
        except:
            pass
    print('Rename Done!')

In [40]:
renamefile(df_copyfile)

Rename Done!


In [15]:
# 遍歷資料夾(D:/TuneFab Spotify Music Converter/songs2)，查看需要下載的歌詞
def through_path(yourPath):
    
    allFileList = os.listdir(yourPath)
    dir_artist = [] # 儲存歌手的list
    dir_artist2 = [] # 儲存歌手的list(寫入dataframe用)
    dir_artist3 = [] # 儲存歌手原始名稱的list(處理資料夾名稱有多位歌手的情況)
    dir_title = [] # 儲存歌名的list

    for file in allFileList: # 遍歷songs資料夾
        if os.path.isdir(os.path.join(yourPath,file)):
            print("dir: " + file) # 列出songs資料夾底下有那些歌手
            dir_artist.append(file)

            artistPath = yourPath + '/' + file # 形式：D:/TuneFab Spotify Music Converter/songs/{歌手名稱}
            #print(artistPath)
            artistFileList = os.listdir(artistPath)
            #print(artistFileList)

            for file2 in artistFileList: # 遍歷每位歌首的資料夾
                songPath = artistPath + '/' + file2
                if os.path.isfile(songPath): # 形式：D:/TuneFab Spotify Music Converter/songs/{歌手名稱}/{歌曲名稱}
                    #print("dir2: " + file2)
                    file3 = file2.replace('.mp3','').replace("’","'") #取代副檔名
                    dir_title.append(file3)
                    
#                     若遇到檔案有特殊符號，如「’」
#                     new = file2.replace("’","'")
#                     mp3filename_edit = artistPath + '/' + new
#                     os.rename(songPath,mp3filename_edit)
                
                    name = songPath.split('/')[-2] # 取songPath位址倒數第二個值
                    name_list = name.split(',') 
                    dir_artist2.append(name_list[0]) # 針對資料夾有多位歌手處理，取第一個為主要演唱者
                    
                    # 原始歌手資料夾名稱
                    originalname = artistPath.split('/')[-1]
                    dir_artist3.append(originalname)

        #print('-'*80)

    # 輸出datafrme(df_songlist)和xlsx(df_songlist.xlsx)
    df_songlist = pd.DataFrame(list(zip(dir_artist3,dir_artist2,dir_title)),columns=['Original','Artist','Title'])
    #df_songlist_select = df_songlist.loc[~df_songlist['Title'].str.contains(string)].reset_index(drop=True)
    df_songlist.to_excel('table/df_songlist.xlsx',index=0)
    return df_songlist

In [16]:
yourPath = r'D:/TuneFab Spotify Music Converter/songs2' # mp3資料夾位址
df_songlist_select = through_path(yourPath)
df_songlist_select

dir: 5 Seconds of Summer
dir: Aaron Carter
dir: Aaron Carter,Nick Carter,No Secrets
dir: Adele
dir: Agnetha Fältskog,Ola Håkansson


,Original,Artist,Title
0,5 Seconds of Summer,5 Seconds of Summer,18
1,5 Seconds of Summer,5 Seconds of Summer,Airplanes
2,5 Seconds of Summer,5 Seconds of Summer,Amnesia
3,5 Seconds of Summer,5 Seconds of Summer,Babylon
4,5 Seconds of Summer,5 Seconds of Summer,Beside You
...,...,...,...
142,Adele,Adele,Set Fire to the Rain
143,Adele,Adele,Someone Like You
144,Adele,Adele,Take It All
145,Adele,Adele,Turning Tables


In [6]:
df_songlist_select = through_path()

for i in range (len(df_songlist_select)):
    artist = df_songlist_select['Artist'][i]
    title = df_songlist_select['Title'][i]
    lyrics = print_lyrics(artist,title)
    filename = save2txt(artist,title,lyrics)
    print('Filename: '+ filename)
    #print('Lyrics:'+ '\n' + lyrics)

dir: 5 Seconds of Summer
dir: Adele
Filename: lyrics/5 Seconds of Summer_18.txt
Filename: lyrics/5 Seconds of Summer_Airplanes.txt
Filename: lyrics/5 Seconds of Summer_Amnesia.txt
Filename: lyrics/5 Seconds of Summer_Babylon.txt
Filename: lyrics/5 Seconds of Summer_Beside You.txt
Filename: lyrics/5 Seconds of Summer_Better Man.txt
Filename: lyrics/5 Seconds of Summer_Broken Home.txt
Filename: lyrics/5 Seconds of Summer_Castaway.txt
Filename: lyrics/5 Seconds of Summer_Catch Fire.txt
Filename: lyrics/5 Seconds of Summer_Close As Strangers.txt
Filename: lyrics/5 Seconds of Summer_Disconnected.txt
Filename: lyrics/5 Seconds of Summer_Don t Stop.txt
Filename: lyrics/5 Seconds of Summer_Empty Wallets.txt
Filename: lyrics/5 Seconds of Summer_End Up Here.txt
Filename: lyrics/5 Seconds of Summer_English Love Affair.txt
Filename: lyrics/5 Seconds of Summer_Everything I Didn t Say.txt
Filename: lyrics/5 Seconds of Summer_Fly Away.txt
Filename: lyrics/5 Seconds of Summer_Ghost Of You.txt
Filename

In [23]:
# 獲取歌詞額外方案
# https://github.com/fr31/spotifylyrics/blob/master/lyrics.py
def LyricWikia(artist, title):
    proxy = urllib.request.getproxies()
    url = 'http://lyrics.wikia.com/api.php?action=lyrics&artist={artist}&song={title}&fmt=json&func=getSong'.format(artist=artist,
                                                                                                                    title=title).replace(" ","%20")
    r = requests.get(url, timeout=15, proxies=proxy)
    # We got some bad formatted JSON data... So we need to fix stuff :/
    returned = r.text
    returned = returned.replace("\'", "\"")
    returned = returned.replace("song = ", "")
    returned = json.loads(returned)
    if returned["lyrics"] != "Not found":
        # set the url to the url we just recieved, and retrieving it
        r = requests.get(returned["url"], timeout=15, proxies=proxy)
        soup = BeautifulSoup(r.text, 'html.parser')
        soup = soup.find("div", {"class": "lyricbox"})
        [elem.extract() for elem in soup.findAll('div')]
        [elem.replaceWith('\n') for elem in soup.findAll('br')]
        #with old BeautifulSoup the following is needed..? For recent versions, this isn't needed/doesn't work
        try:
            #soup = BeautifulSoup(str(soup), convertEntities=BeautifulSoup.HTML_ENTITIES)
            soup = BeautifulSoup(str(soup), 'html.parser')
        except:
            pass
        soup = BeautifulSoup(re.sub(r'(<!--[.\s\S]*-->)', '', str(soup)), 'html.parser')
        [elem.extract() for elem in soup.findAll('script')]
        return(soup.getText())
    else:
        return("error")

In [30]:
artist = "Haley Reinhart"
title = "These Boots Are Made For Walkin\'"
LyricWikia(artist, title)

'error'